<a href="https://colab.research.google.com/github/pushkarsingh777/Keras-tuner/blob/main/Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

In [2]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.9 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

<ipython-input-3-37a28a73e261>:5: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [4]:
df = pd.read_csv('Real_Combine.csv')
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [5]:
df.isnull().sum()

,0
T,0
TM,0
Tm,0
SLP,0
H,0
VV,0
V,0
VM,0
PM 2.5,1


In [6]:
df.dropna(inplace=True)

In [7]:
df.isnull().sum()

,0
T,0
TM,0
Tm,0
SLP,0
H,0
VV,0
V,0
VM,0
PM 2.5,0


In [8]:
def build_model(hp):
  model = keras.Sequential()
  for i in range(hp.Int('num_layers',2,10)):
    model.add(layers.Dense(units = hp.Int('units_'+str(i),min_value = 32,max_value = 256,step = 32), activation='relu'))
  model.add(layers.Dense(1,activation='linear'))
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',[1e-2,1e-3,1e-4])),loss='mean_absolute_error',metrics=['mean_absolute_error'])
  return model

In [9]:
tuner = RandomSearch(build_model,
                     objective='val_mean_absolute_error',
                     max_trials=10,
                     executions_per_trial=3,
                     directory='project',
                     project_name='Air Quality Index',
                     max_consecutive_failed_trials=5,
                     max_retries_per_trial=2)

In [10]:
tuner.search_space_summary

<bound method BaseTuner.search_space_summary of <keras_tuner.src.tuners.randomsearch.RandomSearch object at 0x7b48db17d720>>

In [11]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

In [14]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 10 Complete [00h 00m 16s]
val_mean_absolute_error: 63.94311650594076

Best val_mean_absolute_error So Far: 49.71299489339193
Total elapsed time: 00h 02m 26s


In [15]:
tuner.results_summary()

Results summary
Results in project/Air Quality Index
Showing 10 best trials
Objective(name="val_mean_absolute_error", direction="min")

Trial 03 summary
Hyperparameters:
num_layers: 3
units_0: 192
units_1: 192
learning_rate: 0.01
units_2: 192
units_3: 128
units_4: 160
units_5: 224
units_6: 96
units_7: 96
units_8: 64
units_9: 32
Score: 49.71299489339193

Trial 04 summary
Hyperparameters:
num_layers: 8
units_0: 128
units_1: 160
learning_rate: 0.01
units_2: 96
units_3: 96
units_4: 32
units_5: 64
units_6: 256
units_7: 96
units_8: 256
units_9: 256
Score: 52.10174051920573

Trial 06 summary
Hyperparameters:
num_layers: 5
units_0: 192
units_1: 160
learning_rate: 0.001
units_2: 64
units_3: 160
units_4: 256
units_5: 192
units_6: 96
units_7: 96
units_8: 32
units_9: 64
Score: 56.581817626953125

Trial 05 summary
Hyperparameters:
num_layers: 2
units_0: 256
units_1: 224
learning_rate: 0.001
units_2: 32
units_3: 64
units_4: 96
units_5: 64
units_6: 128
units_7: 192
units_8: 192
units_9: 32
Score: 58.